<a href="https://colab.research.google.com/github/Phimphika113/sobeam_year4/blob/main/weekly_work_neos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyomo ## ติดตั้ง Pyomo
from pyomo.environ import *
from pyomo import environ as pe

!apt-get install -y glpk-utils  ##Solver
solver = pe.SolverFactory('glpk', executable='/user/bin/glpsol')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 383.3 kB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]

Failed to set executable for solver glpk. File with name=/user/bin/glpsol either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/base/solvers.py", line 148, in __call__
    opt = self._cls[_name](**kwds)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/solvers/plugins/solvers/GLPK.py", line 92, in __init__
    SystemCallSolver.__init__(self, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/solver/shellcmd.py", line 66, in __init__
    self.set_executable(name=executable, validate=validate)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/solver/shellcmd.py", line 115, in set_executable
    raise ValueError(
ValueError: Failed to set executable for solver glpk. File with name=/user/bin/glpsol either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.


In [2]:
from pyomo.opt import SolverFactory

#คำอธิบายปัญหา
ร้านอาหารแห่งใหม่ได้เปิดขึ้นภายในบริเวณมหาวิทยาลัย ซึ่งจะเปิดให้บริการตลอด 24 ชั่วโมง 7 วันต่อสัปดาห์ โดยในแต่ละวันจะมี 3 กะ กะละ 8 ชั่วโมง กะเช้าคือ 06.00-14.00 น. กะเย็นคือ 14.00-22.00 น. และกะกลางคืนคือ 22.00-06.00 น. ของวันถัดไป

ในเวลากลางคืนจะมีคนงานเพียงคนเดียว ในขณะที่ในเวลากลางวันจะมีสองคน ยกเว้นในวันอาทิตย์ซึ่งจะมีเพียงคนเดียวต่อกะ คนงานแต่ละคนจะต้องทำงานไม่เกิน 40 ชั่วโมงต่อสัปดาห์ และต้องพักผ่อน 12 ชั่วโมงระหว่างกะงานทั้งสอง

สำหรับวันหยุดประจำสัปดาห์ พนักงานที่หยุดวันอาทิตย์วันหนึ่งก็มักจะเลือกหยุดในวันเสาร์วันเดียวกันด้วย

ตามหลักการแล้วมีพนักงานอยู่สิบคน ซึ่งถือว่ามากเกินไปอย่างเห็นได้ชัด ยิ่งพนักงานต้องการน้อยลงเท่าไร ทรัพยากรสำหรับร้านอื่น ๆ ก็ยิ่งมากขึ้นเท่านั้น

In [3]:
# Define days (1 week)
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Enter shifts of each day
shifts = ['morning', 'evening', 'night']  # 3 shifts of 8 hours
days_shifts = {day: shifts for day in days}  # dict with day as key and list of its shifts as value

# Enter workers ids (name, number, ...)
workers = ['W' + str(i) for i in range(1, 11)]  # 10 workers available, more than needed


In [4]:
# Initialize model
model = ConcreteModel()

# binary variables ที่แสดงว่าพนักงานได้รับการกำหนดเวลาไว้ที่ใด
# ตัวแปรไบนารีที่จัดทำดัชนี พนักงาน, วัน, กะ โดน 1 คือคนงานต้องทำงานกะนั้นในวันนั้น
model.works = Var(((worker, day, shift) for worker in workers for day in days for shift in days_shifts[day]),
                  within=Binary, initialize=0)

# binary variables ที่แสดงว่าจำเป็นต้องมีคนงานหรือไม่
#  1 = หากจำเป็นต้องรวมคนงานไว้ในกำลังแรงงาน
model.needed = Var(workers, within=Binary, initialize=0)

# binary variables ที่แสดงว่าคนงานทำงานในวันอาทิตย์แต่ไม่ทำงานในวันเสาร์ (หลีกเลี่ยงหากเป็นไปได้)
# 1 = ถ้าไม่ทำงานในวันอาทิตย์แต่กลับทำงานในวันเสาร์
model.no_pref = Var(workers, within=Binary, initialize=0)

#วัตถุประสงค์
- คือค้นหาตารางงานที่ลดจำนวนคนงานให้เหลือน้อยที่สุด
-และเมื่อทำได้แล้ว จำนวนคนงานที่ทำงานในวันอาทิตย์แต่ไม่ใช่ในวันเสาร์ก็รวมอยู่ด้วย
- *คูณส่วนของจำนวนคนงานด้วยค่าคงที่ที่มากพอ*
*เพื่อให้พิจารณาลดความต้องการทำงานในวันหยุดสุดสัปดาห์ให้เหลือน้อยที่สุดหลังจากตัดสินใจเลือกจำนวนคนงานที่เหมาะสมที่สุดแล้วเท่านั้น*

In [5]:
# Define an objective function with model as input, to pass later
def obj_rule(m):
    c = len(workers)
    return sum(m.no_pref[worker] for worker in workers) + sum(c * m.needed[worker] for worker in workers)
# รวมจำนวนคนงานที่่ทำเสารื + รวมจำนวนคนงานที่จำเป็น(ไม่่เกิน10) จะต้องน้อยที่สุด
# เนื่องจาก sum(m.no_prefer) มีค่าไม่เกิน len(workers) ดังนั้น len(workers) + 1 จึงเป็นค่าคงที่ที่ถูกต้อง

# add objective function to the model. rule (pass function) or expr (pass expression directly)
model.obj = Objective(rule=obj_rule, sense=minimize)

In [6]:
model.constraints = ConstraintList()  # Create a set of constraints

# Constraint: กะงานทั้งหมดได้รับการมอบหมาย
for day in days:
    for shift in days_shifts[day]:
        if day in days[:-1] and shift in ['morning', 'evening']:
            # กะวันธรรมดาและวันเสาร์มีคนงานแค่สองคนเท่านั้น
            model.constraints.add(  # to add a constraint to model.constraints set
                2 == sum(model.works[worker, day, shift] for worker in workers)
            )
        else:
            # กะวันอาทิตย์และกะกลางคืนมีคนงานแค่ 1 คนเท่านั้น
            model.constraints.add(
                1 == sum(model.works[worker, day, shift] for worker in workers)
            )

# Constraint: ทำงานไม่เกิน 40 ชั่วโมง
for worker in workers:
    model.constraints.add(
        40 >= sum(8 * model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    )

# Constraint: เวลาพักระหว่างกะงานสองกะคือ 12 ชั่วโมง (ก้คือ อย่างน้อยสองกะงาน)
for worker in workers:
    for j in range(len(days)):
        # if working in morning, cannot work again that day
        model.constraints.add(
            1 >= sum(model.works[worker, days[j], shift] for shift in days_shifts[days[j]])
        )
        # ถ้าทำงานตอนบ่าย จะทำบ่ายของวันถัดไป (ปล. หลังวันอาทิตย์ให้เลื่อนไปทำงานวันจันทร์หน้า)
        model.constraints.add(
            1 >= sum(model.works[worker, days[j], shift] for shift in ['evening', 'night']) +
            model.works[worker, days[(j + 1) % 7], 'morning']
        )
        # if working in night, until next night
        model.constraints.add(
            1 >= model.works[worker, days[j], 'night'] +
            sum(model.works[worker, days[(j + 1) % 7], shift] for shift in ['morning', 'evening'])
        )

# Constraint (def of model.needed)
for worker in workers:
    model.constraints.add(
        10000 * model.needed[worker] >= sum(model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    )  # if any model.works[worker, ·, ·] non-zero, model.needed[worker] must be one; else is zero to reduce the obj function
    # 10000 is to remark, but 5 was enough since max of 40 hours yields max of 5 shifts, the maximum possible sum
#งง ทำไมผลรวมที่เป็นได้ไมเป็น 56 ชั่วโมง หรือ 7 ผลัด

# Constraint (def of model.no_pref)
for worker in workers:
    model.constraints.add(
        model.no_pref[worker] >= sum(model.works[worker, 'Sat', shift] for shift in days_shifts['Sat'])
        - sum(model.works[worker, 'Sun', shift] for shift in days_shifts['Sun'])
    )  ) # ถ้าไม่ทำงานในวันอาทิตย์แต่ทำงานในวันเสาร์ จำเป็นต้องเป็น 1 ไม่เช่นนั้นจะเป็นศูนย์เพื่อลดฟังก์ชัน obj

In [7]:
##!pip install glpk  # Install the glpk solver

##import os
##import pyomo.environ as pe
##from pyomo.opt import SolverFactory, SolverManagerFactory

# ... (rest of your code)

# Set the NEOS email address directly in the environment
##os.environ['NEOS_EMAIL'] = "your_actual_email@example.com"  # Replace with your actual email

#opt = SolverFactory('cbc')  # Select solver
# Try to automatically find the glpk executable
##solver_manager = pe.SolverFactory('glpk')
# If the above fails, you might need to specify the correct path
# solver_manager = pe.SolverFactory('glpk', executable='/path/to/glpsol')  # Replace with the actual path
#solver_manager = SolverManagerFactory('neos')  # Solve in neos server
##results = solver_manager.solve(model)

In [8]:
import os
# Set the NEOS email address directly in the environment
os.environ['NEOS_EMAIL'] = "your_actual_email@example.com"  # Replace with your actual email

opt = SolverFactory('cbc')  # Select solver
solver_manager = SolverManagerFactory('neos')  # Solve in neos server
##solver_manager = pe.SolverFactory('glpk', executable='/user/bin/glpsol')
results = solver_manager.solve(model, opt=opt)

In [9]:
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 261
  Number of variables: 230
  Sense: unknown
Solver: 
- Status: ok
  Message: CBC 2.10.10 optimal, objective 72; 0 nodes, 88 iterations, 0.108329 seconds
  Termination condition: optimal
  Id: 0
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



#Extract the solution
ต้องการทราบว่าวิธีใดไม่เป็นศูนย์ และเพื่อให้เห็นภาพได้ เราต้องประมวลผลวิธีแก้ปัญหาที่กำหนดให้

In [10]:
def get_workers_needed(needed):
    """Extract to a list the needed workers for the optimal solution."""
    workers_needed = []
    for worker in workers:
        if needed[worker].value == 1:
            workers_needed.append(worker)
    return workers_needed


def get_work_table(works):
    """Build a timetable of the week as a dictionary from the model's optimal solution."""
    week_table = {day: {shift: [] for shift in days_shifts[day]} for day in days}
    for worker in workers:
        for day in days:
            for shift in days_shifts[day]:
                    if works[worker, day, shift].value == 1:
                        week_table[day][shift].append(worker)
    return week_table


def get_no_preference(no_pref):
    """Extract to a list the workers not satisfied with their weekend preference."""
    return [worker for worker in workers if no_pref[worker].value == 1]


workers_needed = get_workers_needed(model.needed)  # dict with the optimal timetable
week_table = get_work_table(model.works)  # list with the required workers
workers_no_pref = get_no_preference(model.no_pref)  # list with the non-satisfied workers (work on Sat but not on Sun)


In [11]:
import json

In [12]:
print('Workers needed:')
[print('  ' + worker) for worker in workers_needed];

Workers needed:
  W2
  W3
  W5
  W7
  W8
  W9
  W10


## ตรวจสอบ
- ตรวจสอบโซลูชันที่ได้รับ
- ตรวจสอบว่าเงื่อนไขที่เสนอทั้งหมดเป็นไปตามข้อกำหนดหรือไม่
*หากบันทึกพจนานุกรมของตารางเวลาที่เหมาะสมที่สุดในรูปแบบ JSON โซลูชันจะมีลักษณะดังนี้:*

In [13]:
print('Work schedule:')
print(json.dumps(week_table, indent=2))

Work schedule:
{
  "Mon": {
    "morning": [
      "W3",
      "W9"
    ],
    "evening": [
      "W8",
      "W10"
    ],
    "night": [
      "W2"
    ]
  },
  "Tue": {
    "morning": [
      "W3",
      "W5"
    ],
    "evening": [
      "W8",
      "W10"
    ],
    "night": [
      "W9"
    ]
  },
  "Wed": {
    "morning": [
      "W5",
      "W7"
    ],
    "evening": [
      "W2",
      "W10"
    ],
    "night": [
      "W3"
    ]
  },
  "Thu": {
    "morning": [
      "W7",
      "W9"
    ],
    "evening": [
      "W8",
      "W10"
    ],
    "night": [
      "W2"
    ]
  },
  "Fri": {
    "morning": [
      "W7",
      "W9"
    ],
    "evening": [
      "W3",
      "W10"
    ],
    "night": [
      "W5"
    ]
  },
  "Sat": {
    "morning": [
      "W2",
      "W7"
    ],
    "evening": [
      "W8",
      "W9"
    ],
    "night": [
      "W3"
    ]
  },
  "Sun": {
    "morning": [
      "W7"
    ],
    "evening": [
      "W8"
    ],
    "night": [
      "W2"
    ]
  }
}


In [14]:
print('Workers not satisfied by weekend condition:')
[print('  ' + worker) for worker in workers_no_pref];

Workers not satisfied by weekend condition:
  W3
  W9


In [15]:
print('Objective value:', model.obj())

Objective value: 72.0


In [16]:
solver = pe.SolverFactory('glpk', executable='/user/bin/glpsol')

Failed to set executable for solver glpk. File with name=/user/bin/glpsol either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/base/solvers.py", line 148, in __call__
    opt = self._cls[_name](**kwds)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/solvers/plugins/solvers/GLPK.py", line 92, in __init__
    SystemCallSolver.__init__(self, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/solver/shellcmd.py", line 66, in __init__
    self.set_executable(name=executable, validate=validate)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/solver/shellcmd.py", line 115, in set_executable
    raise ValueError(
ValueError: Failed to set executable for solver glpk. File with name=/user/bin/glpsol either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.
